# 0. Preamble

In [1]:
import os
import datetime
import logging
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import thomas.core

In [2]:
logging_setup = False

# Functions below based on 
# https://stackoverflow.com/questions/12544056/how-do-i-get-the-current-ipython-jupyter-notebook-name
def get_kernel_id():
    import ipykernel
    
    connection_file = os.path.basename(ipykernel.get_connection_file())
    return connection_file.split('-', 1)[1].split('.')[0]

def notebook_path():
    """Returns the absolute path of the Notebook or None if it cannot be determined
    NOTE: works only when the security is token-based or there is also no password
    """
    from notebook import notebookapp
    import urllib
    import json

    kernel_id = get_kernel_id()
    
    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  # No token and no password, ahem...
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+'api/sessions?token='+srv['token'])
            sessions = json.load(req)
            for sess in sessions:
                if sess['kernel']['id'] == kernel_id:
                    return os.path.join(srv['notebook_dir'],sess['notebook']['path'])
        except:
            pass  # There may be stale entries in the runtime directory 
        
    return None

In [3]:
print()
print('available imports:')
print('  import os')
print('  import logging')
print('  import pandas as pd')
print('  import numpy as np')
print()

available imports:
  import os
  import logging
  import pandas as pd
  import numpy as np



In [4]:
__kernel_id = get_kernel_id()
print(f'connect to this kernel with:\n  jupyter console --existing {get_kernel_id()}')
print()

connect to this kernel with:
  jupyter console --existing 5adde929-5608-440f-b3ef-e38428812bf5


In [5]:
os.path.split(notebook_path())

('/Users/melle/python_projects/thomas/thomas-core/notebooks',
 '0. Preamble.ipynb')

In [6]:
if logging_setup == False:
    log = logging.getLogger()

    # handler = OutputWidgetHandler()
    pathname, filename = os.path.split(notebook_path())
    pathname = os.path.normpath(os.path.join(pathname, '../logs'))
    
    try:
        os.makedirs(pathname)
    except:
        print(f'Could not create logging directory "{pathname}"')
    
    logfile = os.path.splitext(filename)[0] + '.log'
    logfile = os.path.join(pathname, logfile)
    print(f'Logging to: "{logfile}"')
    
    # Delete the logfile to start fresh.
    # Use tail -F to follow the file regardless :-)
    try:
       os.remove(logfile)
    except:
       pass

    with open(logfile, 'w') as fp:
        fp.write('\n')
    
    handler = logging.FileHandler(logfile)
    handler.setFormatter(
        logging.Formatter(
            '%(asctime)s  - %(name)-14s - [%(levelname)s] %(message)s',
            "%Y-%m-%d %H:%M:%S")
    )
    log.addHandler(handler)
    log.setLevel(logging.DEBUG)
    
    logging.getLogger('parso.cache').setLevel(logging.WARNING)
    logging.getLogger('parso.python.diff').setLevel(logging.WARNING)
    logging.getLogger('urllib3.connectionpool').setLevel(logging.WARNING)
    logging.getLogger('matplotlib').setLevel(logging.WARNING)
    logging.getLogger('matplotlib.font_manager').setLevel(logging.WARNING)
    
    logging_setup = True
    
    # Clean up
    del fp, logfile


log.info('-' * 60)
log.info(f'Starting notebook "{os.path.split(notebook_path())[-1] }"')
log.info(f'thomas.core version: {thomas.core.__version__}')
log.info(f'Pandas version: {pd.__version__}')
log.info(f'Current working directory: "{os.getcwd()}"')
log.info('-' * 60)

In [ ]:
print(f'Current date/time: {datetime.datetime.now().strftime("%d-%m-%Y, %H:%M")}')
print(f'Current working directory: "{os.getcwd()}"')

In [7]:
# handler.clear_logs()
# for x in range(200):
#     log.debug(f'{x:02}: yeaj')